# Volvo Trucks Analytics

Imports and global variables go here

In [4]:
import pandas as pd
import numpy as np
import csv
truck1csv = "../data/trucks/truck1.csv"
truck2csv = "../data/trucks/truck2.csv"
rowNaNThreshold = 41

Define all cleaning methods here

In [5]:
def readCsv (truck):
    return pd.read_csv(truck, header=[0])
    
def cullUtcCols (truckData):
    noUTC = truckData.drop(["UTC hour", "UTC minute", "UTC second", "UTC month", "UTC day", "UTC year"], axis = 1)
    return noUTC

def removeUnnecessaryRows(dataFrame):
    return dataFrame.dropna(thresh=rowNaNThreshold)

Cleaning of Truck 1 Data starts here

In [6]:
truck1data = readCsv(truck1csv)
truck1data = cullUtcCols(truck1data)
truck1data = removeUnnecessaryRows(truck1data)

# After cleaning, check the shape of the dataframe
truck1data.shape

FileNotFoundError: [Errno 2] File b'../data/trucks/truck1.csv' does not exist: b'../data/trucks/truck1.csv'

Cleaning of Truck 2 Data starts here

In [4]:
truck2data = readCsv(truck2csv)
truck2data = cullUtcCols(truck2data)
truck2data = removeUnnecessaryRows(truck2data)

# After cleaning, check the shape of the dataframe
truck2data.shape

(1686323, 40)